In [19]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [16]:
import mysql.connector

In [17]:
def connectToDatabase(host,user,password,database):
    connector = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    return connector

In [18]:
connection = connectToDatabase("localhost","root","root","c361")

In [19]:
cursor = connection.cursor()

In [20]:
create_table = "create table if not exists example_table(id int primary key auto_increment, name varchar(255), age int)"

In [21]:
try:
    cursor.execute(create_table)
except Exception as e:
    print(e)
    print("Unsuccessful connection to database")

In [22]:
insert_data = "insert into example_table(name,age) values(%s,%s)"

In [33]:
cursor.execute(insert_data,["John",26])

In [45]:
# execute many
data = [
    ("John",26),
    ("Jane",25),
    ("Jack",24)
]

cursor.executemany(insert_data,data)

In [23]:
connection.commit()

In [24]:
select_data = "select * from example_table"

In [25]:
cursor.execute(select_data)
for i in cursor.fetchall():
    print(i)

(2, 'John', 25)
(3, 'John', 26)
(4, 'John', 26)
(5, 'Jane', 25)
(6, 'Jack', 24)


In [49]:
# using where clause
select_data = "select * from example_table where name = %s"

cursor.execute(select_data,["John"])
for i in cursor.fetchall():
    print(i)

(2, 'John', 25)
(3, 'John', 26)
(4, 'John', 26)


In [50]:
# using where clause with multiple conditions
select_data = "select * from example_table where name = %s and age = %s"

cursor.execute(select_data,["John",26])
for i in cursor.fetchall():
    print(i)
    

(3, 'John', 26)
(4, 'John', 26)


In [51]:
# using group by
select_data = "select name, count(*) from example_table group by name"

cursor.execute(select_data)
for i in cursor.fetchall():
    print(i)

('John', 3)
('Jane', 1)
('Jack', 1)


In [11]:
# mysql join
select_data = "select * from example_table a join example_table b on a.id = b.id+1"

cursor.execute(select_data)
for i in cursor.fetchall():
    print(i)

(3, 'John', 26, 2, 'John', 25)
(4, 'John', 26, 3, 'John', 26)
(5, 'Jane', 25, 4, 'John', 26)
(6, 'Jack', 24, 5, 'Jane', 25)


In [10]:
# alter table
alter_table = "alter table example_table auto_increment=100"

cursor.execute(alter_table)

In [13]:
cursor.execute(insert_data,["Aditya",100])

In [27]:
cursor.execute(select_data)
for i in cursor.fetchall():
    print(i)

(2, 'John', 25)
(3, 'John', 26)
(4, 'John', 26)
(5, 'Jane', 25)
(6, 'Jack', 24)


In [31]:
# example of rollback
insert_data = "insert into example_table(name,age) values(%s,%s)"

cursor.execute(insert_data,["James",26])

# execute many
data = [
    ("Johnny",26),
    ("Jenny",25),
    ("JJ",24)
]

cursor.executemany(insert_data,data)

connection.rollback()

cursor.execute(select_data)
for i in cursor.fetchall():
    print(i)


(2, 'John', 25)
(3, 'John', 26)
(4, 'John', 26)
(5, 'Jane', 25)
(6, 'Jack', 24)


In [32]:
# example  of stored procedure
create_procedure = """
create procedure get_all_data()
begin
select * from example_table;
end
"""

cursor.execute(create_procedure)

cursor.callproc("get_all_data")

for i in cursor.stored_results():
    print(i.fetchall())
    

[(2, 'John', 25), (3, 'John', 26), (4, 'John', 26), (5, 'Jane', 25), (6, 'Jack', 24)]


In [35]:
# example of stored procedure with parameters
create_procedure = """
create procedure if not exists get_data_by_name(in name varchar(255))
begin
select * from example_table where name = name;
end
"""

cursor.execute(create_procedure) 

cursor.callproc("get_data_by_name",["John"])

for i in cursor.stored_results():
    print(i.fetchall())


[(2, 'John', 25), (3, 'John', 26), (4, 'John', 26), (5, 'Jane', 25), (6, 'Jack', 24)]


In [37]:
cursor.close()
connection.close()